In [1]:
import pandas as pd
import numpy as np
from functional import seq

In [43]:
original_df = pd.read_csv("./cache/occupational_category_match.csv")
original_df.head()

,physical_exam_conclusion,physical_exam_conclusion_detail
0,二氯甲烷_职业禁忌证;甲苯_其他疾病或异常;乙酸乙酯_其他疾病或异常;四氢呋喃_其他疾病或异...,二氯甲烷_慢性肝病;甲苯_舒张压异常;乙酸乙酯_舒张压异常;四氢呋喃_舒张压异常;异丙醇_舒...
1,以上未提及的可导致职业病的其他化学因素_职业禁忌证;高温(高温作业)_职业禁忌证;乙酸_其他...,以上未提及的可导致职业病的其他化学因素_慢性肝病;高温(高温作业)_未控制的糖尿病;乙酸_谷...
2,粉尘类_其他疾病或异常;噪声_职业禁忌证;高温(高温作业)_其他疾病或异常,"粉尘类_超重,肝功能轻度升高,肺微小结节影（边界欠清）;噪声_高频段3000Hz，4000H..."
3,二氯甲烷_职业禁忌证;甲苯_其他疾病或异常;乙酸乙酯_其他疾病或异常;四氢呋喃_其他疾病或异...,二氯甲烷_慢性肝病;甲苯_谷丙转氨酶异常;乙酸乙酯_谷丙转氨酶异常;四氢呋喃_谷丙转氨酶异常...
4,二氯甲烷_职业禁忌证;甲苯_其他疾病或异常;乙酸乙酯_其他疾病或异常;四氢呋喃_其他疾病或异...,二氯甲烷_慢性肝病;甲苯_肝胆脾异常;乙酸乙酯_肝胆脾异常;四氢呋喃_肝胆脾异常;异丙醇_肝...


# 拆分为键值对

In [126]:
seq(original_df["physical_exam_conclusion"].values).map(lambda x: seq(x.split(";")).map(lambda x: x.split("_")[1]).set()).flatten().set()

{'其他疾病或异常', '复查', '疑似职业病', '目前未见异常', '职业禁忌证'}

In [71]:
def split_2nd(x):
    if x is not None:
        list_1st = seq(x.split(";")).filter(lambda x: x!="")
        try:
            res = list_1st.map(lambda x: (x.split("_")[0], x.split("_")[1]) if len(x.split("_"))==2 else None).filter(lambda x: x is not None).dict()
            return res
        except IndexError:
            print(x)
    else:
        return None

In [72]:
original_df["physical_exam_conclusion_dict"] = original_df["physical_exam_conclusion"].apply(split_2nd)
original_df["physical_exam_conclusion_detail_dict"] = original_df["physical_exam_conclusion_detail"].apply(split_2nd)

# 匹配相同键，拼接值为新的字典

如需保留具体危害因素以进行更多分析，需加入harzard字段

In [82]:
def type_match(row):
    dict1 = row["physical_exam_conclusion_dict"]
    dict2 = row["physical_exam_conclusion_detail_dict"]
    merged_dict = {}
    common_keys = set(dict1.keys()).intersection(dict2.keys())
    for key in common_keys:
        # merged_dict[key] = (dict1[key], dict2[key])
        merged_dict.update({dict1[key]:dict2[key]})
    return merged_dict

In [84]:
disease_se = original_df.apply(type_match, axis=1)
disease_se

0                    {'职业禁忌证': '慢性肝病', '其他疾病或异常': '舒张压异常'}
1             {'职业禁忌证': '慢性肝病', '其他疾病或异常': '谷草转氨酶（AST）异常'}
2        {'其他疾病或异常': '超重,肝功能轻度升高,肺微小结节影（边界欠清）', '职业禁忌证'...
3                  {'职业禁忌证': '慢性肝病', '其他疾病或异常': '谷丙转氨酶异常'}
4                    {'职业禁忌证': '慢性肝病', '其他疾病或异常': '肝胆脾异常'}
                               ...                        
40507    {'职业禁忌证': '除噪声外各种原因引起的永久性感音神经性听力损失（500Hz，l000H...
40508    {'职业禁忌证': '各种原因引起永久性感音神经性听力损失（500Hz、1000Hz和200...
40509    {'职业禁忌证': '各种原因引起永久性感音神经性听力损失（500Hz、1000Hz和200...
40510    {'职业禁忌证': '各种原因引起永久性感音神经性听力损失（500Hz、1000Hz和200...
40511                  {'职业禁忌证': '任一耳传导性耳聋，平均语频听力损失≥41dB'}
Length: 40512, dtype: object

# 去重，保留职业禁忌症进行关键词归类

In [100]:
final_res = []
for object in disease_se:
    disease_desc = seq(object.items()).filter(lambda x: x[0]=="职业禁忌证").map(lambda x: x[1])[0]
    # print(disease_desc)
    final_res.append(disease_desc)
final_res = set(final_res)

In [112]:
print(seq(final_res).filter(lambda x: "神经" in x and "神经性听力损失" not in x))

['多发性周围神经病,未控制的糖尿病', '多发性周围神经病', '视网膜及视神经病', '中枢神经系统器质性疾病,高频段3000Hz，4000Hz，6000Hz双耳平均听阈≥40dB', '任一耳传导性耳聋，平均语频听力损失≥41dB,多发性周围神经病', '中枢神经系统及周围神经系统疾病和病史', '中枢神经系统器质性疾病', '腋臭，头癖，泛发性体癣，疥疮，慢性湿疹，神经性皮炎，白癜风，银屑病', '视网膜及视神经病,伴肺功能损害的疾病']


In [113]:
OCCUPATIONAL_DISEASE_TYPE_DICT = {
    "听力": "职业性听力损伤",
    "听阈": "职业性听力损伤",
    "聋": "职业性听力损伤",
    "皮肤": "职业性皮肤病",
    "疹": "职业性皮肤病",
    "血": "职业性心血管系统系统疾病",
    "心脏": "职业性心血管系统系统疾病",
    "心电": "职业性心血管系统系统疾病",
    "呼吸系统": "职业性呼吸系统疾病",
    "肺": "职业性呼吸系统疾病",
    "支气管": "职业性呼吸系统疾病",
    "嗅": "职业性呼吸系统疾病",
    "鼻": "职业性呼吸系统疾病",
    "内分泌": "职业性内分泌系统疾病",
    "甲状腺": "职业性内分泌系统疾病",
    "泌尿": "职业性泌尿生殖系统疾病",
    "糖尿": "职业性泌尿生殖系统疾病",
    "生殖": "职业性泌尿生殖系统疾病",
    "神经系统": "职业性神经系统疾病",
    "周围神经病": "职业性神经系统疾病",
    "视力": "职业性眼病",
    "色": "职业性眼病",
    "盲": "职业性眼病",
    "角膜": "职业性眼病",
    "白内障": "职业性眼病",
    "肝": "职业性中毒性肝病",
    "肾": "职业性中毒性肾病",
    "肿瘤": "职业性肿瘤",
    "放射性": "职业性放射性疾病",
    "骨": "职业性骨关节疾病"
}

# 职业病种类打标测试

In [114]:
import re

def disease_mark(x):
    res = []
    for key, value in OCCUPATIONAL_DISEASE_TYPE_DICT.items():
        if key in x:
            res.append(value)
    return ",".join(set(res))

In [115]:
original_df["physical_exam_conclusion_detail"].apply(disease_mark)

0                          职业性中毒性肝病
1              职业性泌尿生殖系统疾病,职业性中毒性肝病
2        职业性中毒性肝病,职业性听力损伤,职业性呼吸系统疾病
3                          职业性中毒性肝病
4                          职业性中毒性肝病
                    ...            
40507                       职业性听力损伤
40508                       职业性听力损伤
40509                       职业性听力损伤
40510                       职业性听力损伤
40511                       职业性听力损伤
Name: physical_exam_conclusion_detail, Length: 40512, dtype: object

# 功能代码测试

In [7]:
import pandas as pd

# 创建示例数据
data = {'Date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05'],
        'Value': [1, 2, 3, 4, 5]}
df = pd.DataFrame(data)

# 将 'Date' 列转换为日期类型
df['Date'] = pd.to_datetime(df['Date'])

# 按日期进行滚动累加
df['RollingSum'] = df['Value'].cumsum()

# 打印结果
print(df)

        Date  Value  RollingSum
0 2022-01-01      1           1
1 2022-01-02      2           3
2 2022-01-03      3           6
3 2022-01-04      4          10
4 2022-01-05      5          15
